In [2]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import jax.numpy as np
import numpy as onp
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.learners import AR
from timecast.learners._ar import _ar_predict, _ar_batch_window
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/toy_flood/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [3]:
from ealstm.gaip import FloodLSTM
from ealstm.gaip import FloodData
from ealstm.gaip.utils import MSE, NSE

from timecast.optim import SGD
from timecast.learners import AR

cfg_path = "/home/dsuo/src/toy_flood/ealstm/runs/run_2503_0429_seed283956/cfg.json"
flood_data = FloodData(cfg_path)
ea_data = pickle.load(open("../ealstm/runs/run_2503_0429_seed283956/lstm_seed283956.p", "rb"))

In [9]:
LR_AR = 1e-5
AR_INPUT_DIM=32
AR_OUTPUT_DIM=1

results = {}
mses = []
nses = []
sgd = SGD(learning_rate=LR_AR, online=False)

for X, y, basin in tqdm.tqdm(flood_data.generator(), total=len(flood_data.basins)):
    ar = AR(input_dim=AR_INPUT_DIM,
        output_dim=AR_OUTPUT_DIM,
        window_size=flood_data.cfg["seq_length"],
        optimizer=sgd,
        fit_intercept=True,
        constrain=False
       )
    
    # NOTE: difference in indexing convention, so need to pad one row
    X = np.vstack((X[flood_data.cfg["seq_length"]:], np.ones((1, X.shape[1]))))
    Y = y[flood_data.cfg["seq_length"]-1:]
    
    ar.fit([(X, Y, basin)], alpha=0.1)
    ar._history=X[:flood_data.cfg["seq_length"]]
    
    Y_lstm = np.array(ea_data[basin].qsim).reshape(-1, 1)
    Y_target = Y - Y_lstm
    
    Y_ar = ar.predict_and_update(X, Y_target)
    
    Y_hat = Y_lstm + Y_ar
    
    mse = MSE(Y, Y_hat)
    nse = NSE(Y, Y_hat)
    results[basin] = {
        "mse": mse,
        "nse": nse,
        "count": X.shape[0],
        "avg_mse": np.mean(np.array(mses)),
        "avg_nse": np.mean(np.array(nses))
    }
    mses.append(mse)
    nses.append(nse)
    print(basin, mse, nse, np.mean(np.array(mses)), np.mean(np.array(nses)))

01022500 2.0779026 0.5779607 2.0779026 0.5779607
01031500 3.8257647 0.59274423 2.9518337 0.5853525
01047000 5.5681252 0.4782598 3.8239307 0.5496549
01052500 6.206514 0.52608633 4.4195766 0.5437628
01054200 21.59425 0.2673065 7.8545113 0.48847157
01055000 12.730079 0.28079367 8.667106 0.45385858
01057000 3.426944 0.53355014 7.9185114 0.46524307
01073000 3.8199217 0.41978502 7.406188 0.4595608
01078000 2.079548 0.65362835 6.814339 0.48112386
01123000 2.6119509 0.46943247 6.3941 0.47995472
01134500 4.22281 0.48625505 6.19671 0.4805275
01137500 6.8189936 0.32942426 6.248567 0.46793556
01139000 1.893173 0.46846503 5.9135365 0.46797627
01139800 1.7163632 0.52934015 5.6137385 0.47235942
01142500 2.5574992 0.48201257 5.4099894 0.47300297
01144000 2.5001216 0.49140918 5.2281227 0.47415337
01162500 1.4382433 0.71716535 5.0051885 0.4884482
01169000 7.6868253 0.14567012 5.1541686 0.46940497
01170100 5.8601813 0.275007 5.191327 0.4591735
01181000 6.5511646 0.28557032 5.259319 0.45049334
01187300 9.

KeyboardInterrupt: 